In [ ]:
# Imports
import tensorflow as tf
import numpy as np
import pretty_midi
import matplotlib.pyplot as plt

# Load model
model = tf.keras.models.load_model('models/trained_model.h5')

# Generate note function
def Generate_note(notes, model, temperature=1.0):
    inputs = tf.expand_dims(notes, 0)
    predictions = model.predict(inputs)

    pitch_logits = predictions[0]
    pitch = tf.random.categorical(pitch_logits / temperature, num_samples=1)
    pitch = tf.squeeze(pitch, axis=-1)

    step = predictions[1][..., 0]
    duration = predictions[2][..., 0]
    velocity = predictions[3][..., 0]

    return pitch.numpy()[0], step.numpy()[0], duration.numpy()[0], int(velocity.numpy()[0])

# Generate sequence
seed_sequence = train_notes[:25]
generated = []

for _ in range(120):
    pitch, step, duration, velocity = Generate_note(seed_sequence, model, temperature=1.0)
    last_start = seed_sequence[-1][1]
    new_note = [pitch, last_start + step, last_start + step + duration, velocity]
    generated.append(new_note)
    seed_sequence = np.vstack([seed_sequence[1:], new_note])

generated_notes = pd.DataFrame(generated, columns=['pitch', 'start', 'end', 'velocity'])

# Save MIDI
output_pm = construct_midi(generated_notes)
output_pm.write("output/sample1.mid")

# Visualize
Plot_notes(generated_notes)
plot_dist(generated_notes)
